# Ensemble classifier" SVM+DNN"

In [ ]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Define the scope and credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)

# Authenticate and open the Google Sheets document
client = gspread.authorize(credentials)
sheet = client.open('Osteodata').sheet1  

# Get all records from the sheet
records = sheet.get_all_records()

# Convert the records to a pandas DataFrame
df = pd.DataFrame.from_records(records)

# Separate the input features (X) and target class labels (y)
X = df.drop('classification', axis=1)  # Replace 'TargetColumn' with the name of your target column
y = df['classification']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_clf = SVC()
svm_clf.fit(X_train, y_train)

# Train DNN classifier
dnn_clf = MLPClassifier(hidden_layer_sizes=(128, 64))
dnn_clf.fit(X_train, y_train)

# Get predictions from SVM and DNN classifiers
svm_predictions = svm_clf.predict(X_test)
dnn_predictions = dnn_clf.predict(X_test)

# Merge predictions using majority voting
merged_predictions = [max(set([svm_pred, dnn_pred]), key=[svm_pred, dnn_pred].count) for svm_pred, dnn_pred in zip(svm_predictions, dnn_predictions)]

# Evaluate the performance of the merged classifier
accuracy = accuracy_score(y_test, merged_predictions)
print("Merged Classifier Accuracy:", accuracy)
